# Dimentionality Reduction

## Some background info:

1. Network on the **indicator level**

2. **Why** aren we doing dimentionality reduction:   
- Since we've decided to use the correlation coefficients to form weighted ties, the nature of network analysis requires us to have only one variable for each node, that is, select one measure to represent each indicator. Given that there are multiple measures under each indicator, we have to find a way to pick one representative variable for each target. That is, to reduce the dimention to 1.

3. Time Range: 2012 - 2021

4. A major **problem**: **many missing values, sparse data set, thus possibly inaccurate PCA results**
- 2 **solutions**:
    - 1) impute data before composite, and then feed the imputed data into models like PCA, truncated SVD, etc.
    (imputation details: fit a linear
    - ~~2) use models that in some way interpolate missing values by itself, e.g., PPCA~~
    (It turns out PPCA also won't take our original data. It won't take any singular matrix（rank < n, that is, all values missing for some year, e.g., 2021). Still need to impute data before using ppca.)

### Missing value imputation: 
1. the reason of missing: Missing at Random (MAR)

2. Why we choose Linear Interpolation:
What we want to do: Imputation  ->     
What problem: Time Series Problems  ->      
Type of data: Data with trend & without seasonality  ->        
Method: **Linear Interpolation**       
(reference: https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

3. imputed data
in this directory: https://github.com/PeishanLi/G5055_Practicum_Project2/tree/main/Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
# use indicator 3.2.1(imputed) as example(my random choice)
# # data folder paths
# # original data directory （no imputation, some missing values)
# p1 = "/Users/hailey/Documents/GitHub/G5055_Practicum_Project2/Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia"
# # imputed data directory
# p2 = "/Users/hailey/Documents/GitHub/G5055_Practicum_Project2/Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Indonesia"

# in a more general way
indicator = "3.2.1"
country = "Indonesia"
original_prefix = "../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/"
imputed_prefix = "../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/"
original_filepath = f"{original_prefix}{country}/Indicator{indicator}.csv"
imputed_filepath = f"{imputed_prefix}{country}/Indicator{indicator}.csv"
# original_filepath
# tp = "../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia/Indicator3.2.1.csv"
# original_filepath == tp # True
# imputed_filepath
# truepath = "../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Indonesia/Indicator3.2.1.csv"
# imputed_filepath == truepath # True

In [4]:
original_d = pd.read_csv(original_filepath, index_col = "Year")
imputed_d = pd.read_csv(imputed_filepath, index_col = "Year")                      

In [5]:
original_d

,SH_DYN_IMRTN,SH_DYN_MORT,SH_DYN_IMRT,SH_DYN_MORTN
Year,,,,
2012.0,130247.0,31.3,26.0,156767.0
2013.0,125256.0,30.1,25.1,150331.0
2014.0,120213.0,28.9,24.2,143968.0
2015.0,115140.0,27.8,23.3,137566.0
2016.0,110330.0,26.7,22.5,131492.0
2017.0,105617.0,25.7,21.7,125557.0
2018.0,101219.0,24.8,20.9,120070.0
2019.0,97168.0,23.9,20.2,114994.0
2020.0,NaN,NaN,NaN,NaN


In [6]:
imputed_d

,Unnamed: 0,SH_DYN_IMRTN_new,SH_DYN_MORT_new,SH_DYN_IMRT_new,SH_DYN_MORTN_new
Year,,,,,
2012.0,0,130247.000000,31.300000,26.000000,156767.000000
2013.0,1,125256.000000,30.100000,25.100000,150331.000000
2014.0,2,120213.000000,28.900000,24.200000,143968.000000
2015.0,3,115140.000000,27.800000,23.300000,137566.000000
2016.0,4,110330.000000,26.700000,22.500000,131492.000000
2017.0,5,105617.000000,25.700000,21.700000,125557.000000
2018.0,6,101219.000000,24.800000,20.900000,120070.000000
2019.0,7,97168.000000,23.900000,20.200000,114994.000000
2020.0,8,91702.178571,22.632143,19.242857,108038.321429


In [7]:
# remove the column "Year", transform to n-dimentional np array (n is the number of measures)
imputed_d = imputed_d.drop(["Unnamed: 0"], axis = 1)

imputed_d.shape # 10 records of 4 features -- 10 years, 4 measures

(10, 4)

In [8]:
from sklearn.preprocessing import scale
X = pd.DataFrame(scale(imputed_d), index=imputed_d.index, columns=imputed_d.columns)

In [9]:
X

,SH_DYN_IMRTN_new,SH_DYN_MORT_new,SH_DYN_IMRT_new,SH_DYN_MORTN_new
Year,,,,
2012.0,1.596693,1.628351,1.607899,1.602647
2013.0,1.232211,1.234357,1.231503,1.230090
2014.0,0.863931,0.840364,0.855107,0.861758
2015.0,0.493461,0.479203,0.478710,0.491169
2016.0,0.142197,0.118042,0.144136,0.139567
2017.0,-0.201984,-0.210286,-0.190439,-0.203989
2018.0,-0.523161,-0.505782,-0.525013,-0.521611
2019.0,-0.818997,-0.801277,-0.817766,-0.815443
2020.0,-1.218154,-1.217550,-1.218060,-1.218082


## 1. Principal Components Analysis
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space. The input data is centered but not scaled for each feature before applying the SVD.

It uses the LAPACK implementation of the full SVD or a randomized truncated SVD by the method of Halko et al. 2009, depending on the shape of the input data and the number of components to extract.

It can also use the scipy.sparse.linalg ARPACK implementation of the truncated SVD.

Notice that this class **does not support sparse input**. See TruncatedSVD for an alternative with sparse data.

> The model description indicates that it's not a good option for our original dataset (sparse, many missing values)

> Actually, it retures error when dealing with NAs -- ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

> Here I'll try PCA on the imputed data to see whether it works well

In [10]:
from sklearn.decomposition import PCA
# PCA(n_components=None, *, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)
pca = PCA(random_state=2231).fit(X) # n_components is not set, all components are kept

In [11]:
# The loading vectors
pca_loadings = pd.DataFrame(pca.components_.T, index=X.columns, columns=['V1', 'V2', 'V3', 'V4'])
pca_loadings

,V1,V2,V3,V4
SH_DYN_IMRTN_new,0.500004,-0.415259,-0.398548,0.647082
SH_DYN_MORT_new,0.499980,0.854745,-0.088589,0.107624
SH_DYN_IMRT_new,0.500006,-0.205289,0.841338,0.000094
SH_DYN_MORTN_new,0.500010,-0.234152,-0.354205,-0.754786


In [17]:
# Fit the PCA model and transform X to get the principal components
X_plot = pd.DataFrame(pca.fit_transform(X), columns=['PC1', 'PC2', 'PC3', 'PC4'], index=X.index)
X_plot 
# X_plot["PC1"]

In [18]:
# X_plot.index

Float64Index([2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0,
              2020.0, 2021.0],
             dtype='float64', name='Year')

In [56]:
pca.explained_variance_

array([4.44420220e+00, 2.06865128e-04, 3.45797949e-05, 8.03433096e-07])

In [57]:
pca.explained_variance_ratio_ 

array([9.99945494e-01, 4.65446538e-05, 7.78045385e-06, 1.80772446e-07])

> This is really great result! The first principal component explains 99.99% of the total variance, which means it can be a nearly perfect representative or the measures under this indicator.

## 2. TruncatedSVD
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD

Dimensionality reduction using truncated SVD (aka LSA).

This transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). Contrary to PCA, this estimator **does not center the data before computing the singular value decomposition**. This means it can **work with sparse matrices efficiently**.

In particular, truncated SVD works on term count/tf-idf matrices as returned by the vectorizers in sklearn.feature_extraction.text. In that context, it is known as latent semantic analysis (LSA).

This estimator supports two algorithms: a fast randomized SVD solver, and a “naive” algorithm that uses ARPACK as an eigensolver on X * X.T or X.T * X, whichever is more efficient.

~~Since it can work with sparse matrices efficiently, I'm putting in the original data~~(no it didn't work, same error as PCA when it finds NAs)

Still working on imputed data. See if there's any difference between the result from PCA and Truncated SVD.

In [68]:
from sklearn.decomposition import TruncatedSVD
# TruncatedSVD(n_components=2, *, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
# n_components must be < n_features; so here I did n_components=3

# Truncated SVD algorithm 1: a fast randomized SVD solver(default) 
# “randomized” for the randomized algorithm due to Halko (2009).

tsvd1 = TruncatedSVD(n_components=3, algorithm='randomized', n_iter=5, random_state=2231).fit(X)

In [69]:
# The loading vectors
tsvd1_loadings = pd.DataFrame(tsvd1.components_.T, index=X.columns, columns=['V1', 'V2', 'V3'])
tsvd1_loadings

,V1,V2,V3
SH_DYN_IMRTN_new,0.500004,-0.415259,-0.398548
SH_DYN_MORT_new,0.499980,0.854745,-0.088589
SH_DYN_IMRT_new,0.500006,-0.205289,0.841338
SH_DYN_MORTN_new,0.500010,-0.234152,-0.354205


In [70]:
tsvd1.explained_variance_ # different from PCA

array([3.99978198e+00, 1.86178615e-04, 3.11218154e-05])

In [71]:
tsvd1.explained_variance_ratio_ # same result as PCA

array([9.99945494e-01, 4.65446538e-05, 7.78045385e-06])

In [72]:
# Fit the PCA model and transform X to get the principal components
pd.DataFrame(tsvd1.fit_transform(X), columns=['PC1', 'PC2', 'PC3'], index=X.index)

,PC1,PC2,PC3
Year,,,
2012.0,3.217795,0.023438,0.004509
2013.0,2.464080,0.002533,-0.000039
2014.0,1.710580,-0.017784,-0.004570
2015.0,0.971272,-0.008600,-0.010337
2016.0,0.271971,-0.020422,0.004702
2017.0,-0.403349,-0.009006,0.011160
2018.0,-1.037784,0.014848,-0.003645
2019.0,-1.626741,0.014024,-0.001790
2020.0,-2.435923,0.000424,0.000004


In [73]:
# Truncated SVD algorithm 2: ARPACK as an eigensolver on X * X.T or X.T * X
# “arpack” for the ARPACK wrapper in SciPy (scipy.sparse.linalg.svds)
tsvd2 = TruncatedSVD(n_components=3, algorithm='arpack', n_iter=5, random_state=2231).fit(X)

In [74]:
# The loading vectors
tsvd2_loadings = pd.DataFrame(tsvd2.components_.T, index=X.columns, columns=['V1', 'V2', 'V3'])
tsvd2_loadings

,V1,V2,V3
SH_DYN_IMRTN_new,0.500004,-0.415259,-0.398548
SH_DYN_MORT_new,0.499980,0.854745,-0.088589
SH_DYN_IMRT_new,0.500006,-0.205289,0.841338
SH_DYN_MORTN_new,0.500010,-0.234152,-0.354205


In [75]:
tsvd2.explained_variance_ # different from PCA # same as tsvd1

array([3.99978198e+00, 1.86178615e-04, 3.11218154e-05])

In [76]:
tsvd2.explained_variance_ratio_ # same result as PCA

array([9.99945494e-01, 4.65446538e-05, 7.78045385e-06])

In [77]:
# Fit the PCA model and transform X to get the principal components
pd.DataFrame(tsvd2.fit_transform(X), columns=['PC1', 'PC2', 'PC3'], index=X.index)

,PC1,PC2,PC3
Year,,,
2012.0,3.217795,0.023438,0.004509
2013.0,2.464080,0.002533,-0.000039
2014.0,1.710580,-0.017784,-0.004570
2015.0,0.971272,-0.008600,-0.010337
2016.0,0.271971,-0.020422,0.004702
2017.0,-0.403349,-0.009006,0.011160
2018.0,-1.037784,0.014848,-0.003645
2019.0,-1.626741,0.014024,-0.001790
2020.0,-2.435923,0.000424,0.000004


> Since with imputed data there's no difference between PCA and Truncated SVD, to make the most of Truncated SVD and tell the difference between PCA and it, we should imputed data to make it non-sigular(full range) by deleting the empty rows.

> Indicator3.2.1 has no missing value except for the empty rows 2020 and 2021, so I need to try it out on another example. I chose **Indicator2.2.1**.

In [105]:
# in a more general way
indicator = "2.2.1"
country = "Indonesia"
original_prefix = "../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/"
imputed_prefix = "../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/"
original_filepath = f"{original_prefix}{country}/Indicator{indicator}.csv"
imputed_filepath = f"{imputed_prefix}{country}/Indicator{indicator}.csv"

original_d = pd.read_csv(original_filepath, index_col = "Year")
original_d.drop(original_d.index[-1], axis = 0)

,SH_STA_STNT,SH_STA_STNTN
Year,,
2012.0,NaN,8094.2
2013.0,36.4,8077.2
2014.0,NaN,8068.6
2015.0,NaN,8024.6
2016.0,NaN,8068.8
2017.0,NaN,7992.9
2018.0,30.8,7840.4
2019.0,NaN,7668.0
2020.0,NaN,7526.1


In [110]:
# pca = PCA(scale(original_d), random_state=2231).fit(X) 
# # PCA does not work because of missing values:
# # ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [112]:
# tsvd = TruncatedSVD(n_components=3, n_iter=5, random_state=2231).fit(scale(original_d))
# # TSVD does not work because of missing values:
# # ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 3. Probabilistic Principal Components Analysis

### 3.1 ~~GitHub repo: pca-magic~~ (similarly constructed as PCA, but turned out to be ill-defined and yields strange results, so in the end I chose not to use it)           
~~https://github.com/allentran/pca-magic~~

In [79]:
from ppca import PPCA
ppca = PPCA()
# ppca.fit(data=X, d=4, verbose=True)
ppca.fit(X.to_numpy(), d=4) # it won't take pd data frame, only np array

In [80]:
# dir(ppca)
variance_explained = ppca.var_exp
variance_explained # ppca has no attribute like pca.explained_variance_ratio
# stange results

array([1.11105055, 1.11110227, 1.11111091, 1.11111111])

> *Can't set the number of principal component to 1 (as we need)-- LinAlgError: 0-dimensional array given. Array must be at least two-dimensional*
> It's possible to pick the first Component as the representative variable, however, the variance 

In [82]:
# dir(ppca)

In [83]:
components = ppca.data
components

array([[ 1.5966933 ,  1.62835122,  1.60789914,  1.60264667],
       [ 1.23221105,  1.23435745,  1.23150287,  1.23008964],
       [ 0.86393136,  0.84036369,  0.8551066 ,  0.86175831],
       [ 0.49346082,  0.47920274,  0.47871033,  0.49116942],
       [ 0.14219662,  0.11804178,  0.14413587,  0.13956727],
       [-0.20198387, -0.21028636, -0.19043859, -0.20398866],
       [-0.52316057, -0.50578168, -0.52501305, -0.52161148],
       [-0.81899659, -0.80127701, -0.8177657 , -0.81544297],
       [-1.21815405, -1.21755018, -1.21806015, -1.21808234],
       [-1.56619807, -1.56542166, -1.56607733, -1.56610587]])

In [84]:
model_params = ppca.C
model_params

array([[ 5.00004289e-01,  4.15258886e-01, -3.98548301e-01,
        -6.47081927e-01],
       [ 4.99979557e-01, -8.54745271e-01, -8.85888637e-02,
        -1.07624246e-01],
       [ 5.00006158e-01,  2.05288931e-01,  8.41338391e-01,
        -9.38217149e-05],
       [ 5.00009996e-01,  2.34151735e-01, -3.54204710e-01,
         7.54786057e-01]])

### 3.2 The tensorflow version:   
Probabilistic principal components analysis (PCA) is a dimensionality reduction technique that analyzes data via a lower dimensional latent space (Tipping and Bishop 1999). It is often used when there are missing values in the data or for multidimensional scaling.
https://www.tensorflow.org/probability/examples/Probabilistic_PCA
(I've never used tensorflow, I tried but can't make it out. Anyone willing to have a try can read the codes from the link above)

## 4. Random Projection

https://scikit-learn.org/stable/modules/random_projection.html
- Random projection is a powerful dimensionality reduction method that is computationally more efficient tha PCA. It is commonly used in datasets that have **too many dimensions** for PCA to be directly computed.
(We don't need to use it, since PCA works just fine.)

In [114]:
# import numpy as np
# from sklearn import random_projection
# # X = np.random.rand(100, 10000)
# transformer = random_projection.SparseRandomProjection()
# X_new = transformer.fit_transform(X)
# X_new.shape